In [42]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

In [43]:
x = 'xxxapplexxx'
y = 'apple'
x = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
x

'xxx apple xxx'

# 寶典

In [45]:
df1 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl').iloc[:,:-1]
df2 = pd.read_excel('../data/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')
df2.columns = df1.columns
df = df1.append(df2)
df['品名'] = df['品名'].apply(lambda x:x.strip())

# 載入產品csv
val_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO','from']].dropna(axis=0)

# 合併寶典和SPEC
產品集合 = set(df['品名'].values.tolist() + val_df['Y_label'].values.tolist())
產品集合 = set(val_df['Y_label'].values.tolist())
寶典 = [ preprocess(i) for i in 產品集合]
寶典 = sorted(寶典,key=len)[::-1]
寶典[:5]

['PVC SUSPENSION RESIN S 70',
 'PVC SUSPENSION RESIN S65D',
 'PVC SUSPENSION RESIN S65',
 'NYLON HIGH TENACITY YARN',
 'CAUSTIC SODA MICROPEARLS']

# 資料讀取

In [46]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
print(df.shape)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO','from']].dropna(axis=0)
print(df.shape)
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:]))
df['string_X_train'] = df['string_X_train'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(lambda x:x.split('\n'))
df['p'] = 'not find'
for idx in tqdm(df.index):
    for p in df.loc[idx,'Y_label'] + 寶典:
        if p in df.loc[idx,'string_X_train']:
            df.loc[idx,'p'] = p
            continue
print(df[df['p']!='not find'].shape)
df = df[df['p']!='not find']
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)

for i in tqdm(df.index):
    x,y = df.loc[i,'string_X_train'],df.loc[i,'Y_label']
    df.loc[i,'string_X_train'] = x[:x.find(y)] + ' ' + y + ' ' + x[x.find(y)+len(y):]
print(df.shape)
df.head(10)

(20737, 17)
(8570, 4)


<ipython-input-46-b5cdab96cf27>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(df.index):



(6114, 5)


<ipython-input-46-b5cdab96cf27>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):



(6114, 4)


,string_X_train,Y_label,EXPNO,from
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx
7,COMMODITY ...,YARN,25,20210103.xlsx
8,COMMODITY ...,YARN,25,20210103.xlsx
9,COMMODITY ...,YARN,25,20210103.xlsx
10,COMMODITY ...,YARN,25,20210103.xlsx
11,COMMODITY ...,YARN,25,20210103.xlsx
12,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx
13,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx
14,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx


In [48]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
    
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1']!=-1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (6114, 6)
2 (6114, 6)
3 (6114, 6)
4 (6114, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
7,COMMODITY ...,YARN,25,20210103.xlsx,85,89
8,COMMODITY ...,YARN,25,20210103.xlsx,85,89
9,COMMODITY ...,YARN,25,20210103.xlsx,85,89
10,COMMODITY ...,YARN,25,20210103.xlsx,85,89
11,COMMODITY ...,YARN,25,20210103.xlsx,85,89
12,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx,12,23
13,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx,12,23
14,ITEM1 HDPE TAISOX 8010 200MT USD1100MTCONTRA...,TAISOX 8010,18,20210103.xlsx,12,23


In [49]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

<ipython-input-49-4ec65d08a61a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [50]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
  COPPER CLAD LAMINATES    6360 SHEETSAS PER PURCHASE ORDER NO PO0079668 AND PO0081090TRADE TERM CIF PENANG PORT MALAYSIA
COPPER CLAD LAMINATES
1
CIF JAPAN MAIN PORT OR AIRPORT  GLASS FABRICS  
GLASS FABRICS
2
1 90MTS OF ABS RESIN   GRADE   NOAG15A1H  AT USD249000MT  TRADE TERM CIF SHANGHAI CHINA2 504MTS OF ABS RESIN GRADE NOAG15A1H  AT USD249000MT  TRADE TERM CIF DALIAN CHINA3 25MTS OF HIGH IMPACT POLYSTYRENE GRADE NOHP8250  AT USD188000MT  TRADE TERM CIF NINGBO CHINATOTAL AMOUNT USD152606000
GRADE
3
1 100000 KGS OF PP SHEET  RIGID PVC SHEET  A  PET FILM   PET FILM  PP FILM  PP SYNTHETIC PAPER2 80000 PCS OF RIGID PVC PLATE  RIGID PVC SHEET3 100000 KGS OF BOPP FILM  SYNTHETIC PAPER4 800000 SM OF PP SYNTHETIC PAPER  PP FILM  BOPP FILM SYNTHETIC PAPER5 800000 M2 OF PP FILM  PP SHEET  BOPP FILM  SYNTHETIC PAPER PP SYNTHETIC PAPERCIF HONG KONG10 MORE OR LESS IN QUANTITY PER ITEM SHIPPED ALLOWED
PET FILM
4
PVC SUSPENSION RESIN   GRADE   S65S QTY 90 MT PACKED IN 750 KGJUMBO BAG AT THE RATE

In [51]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')
df

(6114, 6)


,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,20210103.xlsx,1,19
5,COMMODITY LLDPE TAISOX 3470 QUANTITY 320MT 2...,TAISOX 3470,18,20210103.xlsx,17,28
7,COMMODITY ...,YARN,25,20210103.xlsx,85,89
8,COMMODITY ...,YARN,25,20210103.xlsx,85,89
9,COMMODITY ...,YARN,25,20210103.xlsx,85,89
...,...,...,...,...,...,...
20723,DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDM...,GRADE,4A,20200712.xlsx,66,71
20724,DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDM...,GRADE,4A,20200712.xlsx,66,71
20725,DESCRIPTION OF GOODS QUANTITYMT UNIT PRICEUSDM...,GRADE,4A,20200712.xlsx,66,71
20731,187 MT PVC RESIN SUSPENSION GRADE S65D OF FO...,GRADE,11,20200712.xlsx,29,34
